In [1]:
import numpy as np 
import math
import random as rd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
def create_data(n_rows, n_cols,choice):
    x_sample=[]
    col=[]
    while len(x_sample)<n_rows:
        if choice:
            col=[rd.randint(-1000,1000) for _ in range(n_cols)]
        else:
            col=[rd.choice([0,1]) for _ in range(n_cols)]
        x_sample.append(col)
    x_sample= np.array(x_sample)
    return x_sample

x = create_data(1000, 42,True)
y= create_data(1000, 1,False)


In [2]:
x_train = x[:700]
x_test=x[700:]
y_train=y[:700]
y_test=y[700:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(700, 42) (700, 1) (300, 42) (300, 1)


In [3]:
# -*- coding: utf-8 -*-
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, Flatten
from keras.models import Model
from keras.layers import Input

def VGG16():
    # Determine proper input shape
    input_shape =(42,1)
    x_input = Input(shape=input_shape)

    # Block 1
    x = Conv1D(64, 3, activation='relu', padding='same', name='block1_conv1')(x_input)
    x = Conv1D(64, 3, activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling1D(2, strides=2, name='block1_pool')(x)

    # Block 2
    x = Conv1D(128, 3, activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv1D(128, 3, activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling1D(2, strides=2, name='block2_pool')(x)

    # Block 3
    x = Conv1D(256, 3, activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv1D(256, 3, activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv1D(256, 3, activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling1D(2, strides=2, name='block3_pool')(x)

    # Block 4
    x = Conv1D(512, 3, activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv1D(512, 3, activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv1D(512, 3, activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling1D(2, strides=2, name='block4_pool')(x)

    # Block 5
    x = Conv1D(512, 3, activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv1D(512, 3, activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv1D(512, 3, activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling1D(2, strides=2, name='block5_pool')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1, activation='sigmoid', name='predictions')(x)  # Sigmoid for binary classification

    #x = GlobalAveragePooling1D()(x)
    #x = GlobalMaxPooling1D()(x)
    #x = Dense(classes, activation='sigmoid', name='predictions')(x)  # Sigmoid for binary classification

    # Create model.
    model = Model(x_input, x, name='vgg16')
    return model
model = VGG16()
lr=0.0001
model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 42, 1)]           0         
                                                                 
 block1_conv1 (Conv1D)       (None, 42, 64)            256       
                                                                 
 block1_conv2 (Conv1D)       (None, 42, 64)            12352     
                                                                 
 block1_pool (MaxPooling1D)  (None, 21, 64)            0         
                                                                 
 block2_conv1 (Conv1D)       (None, 21, 128)           24704     
                                                                 
 block2_conv2 (Conv1D)       (None, 21, 128)           49280     
                                                                 
 block2_pool (MaxPooling1D)  (None, 10, 128)           0     

C:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=30,
    min_lr=0.000001,
    verbose=1,
)
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix

class ConfusionMatrixCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.relist={}
        self.acc=[]

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_binary = (y_pred > 0.5).astype(int)

        cm = confusion_matrix(y_val, y_pred_binary)
        print(f"Confusion Matrix after Epoch {epoch + 1}:\n{cm}")
        print(classification_report(y_test,y_pred_binary))
        acc = float(classification_report(y_test,y_pred_binary).split('\n')[3].split()[2])
        self.acc.append(acc)
        self.relist[acc]=y_pred_binary

# ... Định nghĩa mô hình ...

# Tạo đối tượng callback
confusion_matrix_callback = ConfusionMatrixCallback(validation_data=(x_test,y_test))

callbacks = [early_stopping,lr_scheduler]

checkpointer = ModelCheckpoint(filepath = "Emotion_weights.hdf5", verbose = 1, save_best_only=True)

history = model.fit(x=x_train, y=y_train,validation_data=(x_test,y_test),epochs=20)



Epoch 1/20
22/22 [==============================] - 3s 136ms/step - loss: 0.6962 - accuracy: 0.5014 - val_loss: 0.6930 - val_accuracy: 0.5167
Epoch 2/20
22/22 [==============================] - 2s 78ms/step - loss: 0.6894 - accuracy: 0.5586 - val_loss: 0.7065 - val_accuracy: 0.4867
Epoch 3/20
22/22 [==============================] - 2s 78ms/step - loss: 0.6898 - accuracy: 0.5271 - val_loss: 0.6953 - val_accuracy: 0.5033
Epoch 4/20
22/22 [==============================] - 2s 78ms/step - loss: 0.6636 - accuracy: 0.6071 - val_loss: 0.7091 - val_accuracy: 0.4733
Epoch 5/20
22/22 [==============================] - 2s 82ms/step - loss: 0.6040 - accuracy: 0.6886 - val_loss: 0.7770 - val_accuracy: 0.4900
Epoch 6/20
22/22 [==============================] - 2s 81ms/step - loss: 0.3952 - accuracy: 0.8329 - val_loss: 1.0208 - val_accuracy: 0.4733
Epoch 7/20
22/22 [==============================] - 2s 80ms/step - loss: 0.1997 - accuracy: 0.9200 - val_loss: 1.6891 - val_accuracy: 0.4667
Epoch 8/20
2